### Setup

Run `setup.sh` (once) beforehand

In [ ]:
# Setup torch and other prerequisites
%cd ..
!python -m pip install --upgrade pip
!python -m pip install gdown ipykernel ipywidgets
!python -m pip install torch torchvision torchaudio torchtext torchdata --index-url https://download.pytorch.org/whl/cu118

# Update submodules
!git submodule update --init --recursive

# Create directories
!mkdir data/checkpoints
!mkdir data/dataset/diffusion
!mkdir data/dataset/diffusion_upscaled
!mkdir data/dataset/ngp/train
!mkdir data/dataset/ngp_trained

# Install requirements
!python -m pip install -r requirements.txt

# Setup RealESRGAN
%cd ext/Real-ESRGAN
!python -m pip install -r requirements.txt
!python setup.py develop
%cd ../..

# Setup GroundingDINO + SegmentAnything
%cd ext/Grounded-Segment-Anything
!python -m pip install -q -r requirements.txt
%cd GroundingDINO
!python -m pip install -q .
%cd ../segment_anything
!python -m pip install -q .
%cd ../../..

# Setup InstantNGP
%cd ext/instant-ngp
!apt update
!apt install build-essential git python3-dev python3-pip libopenexr-dev libxi-dev libglfw3-dev libglew-dev libomp-dev libxinerama-dev libxcursor-dev colmap ffmpeg jq
!python -m pip install --upgrade cmake
compute_capability = "70"
%env TCNN_CUDA_ARCHITECTURES=$compute_capability
network_type = "FullyFusedMLP" if int(compute_capability) >= 70 else "CutlassMLP"
print(f"Using {network_type}")
%env NN_CONFIG_PATH = ./configs/nerf/base.json
!jq '.network.otype = "CutlassMLP" | .rgb_network.otype = "CutlassMLP"' $NN_CONFIG_PATH | sponge $NN_CONFIG_PATH
!cmake . -B build -DNGP_BUILD_WITH_GUI=OFF
!cmake --build build --config RelWithDebInfo -j `nproc`
!pip3 install -r requirements.txt
%cd ../..

### Imports

In [ ]:
%cd scripts

In [ ]:
import sys

sys.path.append("..")

import gc
import glob
import json
import os

import cv2
import PIL.Image
from IPython.display import Image

from src.utils.image_wrapper import *

### ControlVideo

In [ ]:
def save_gif(fps, images, scale, gif_path):
    images = [image_wrapper(image, "pil").scale(scale).to_pil() for image in images]
    images[0].save(
        gif_path,
        save_all=True,
        append_images=images[1:],
        optimize=True,
        duration=len(images) / fps,
        loop=0,
    )


def display_gif(gif_path):
    display(Image(data=open(gif_path, "rb").read(), format="png"))

In [ ]:
with open("inference.json", "r") as f:
    conf = json.load(f)

In [ ]:
!python inference_controlvideo.py --settings_path "inference.json" 

In [ ]:
!python inference_realesrgan.py --settings_path "inference.json"

In [ ]:
# Load images
images = sorted(glob.glob(os.path.join(conf["paths"]["upscaled_path"], "*.png")))
images = [PIL.Image.open(image) for image in images]

save_gif(10.0, images, 0.25, "../ipynb/upscaled.gif")
del images
gc.collect()

display_gif("../ipynb/upscaled.gif")

### GroundingDINO + SegmentAnything

In [ ]:
!python inference_groundedsam.py --settings_path inference.json

In [ ]:
# Load masked images
masked_images = sorted(glob.glob(os.path.join(conf["paths"]["masked_path"], "*.png")))
masked_images = [PIL.Image.open(masked) for masked in masked_images]

save_gif(
    10.0,
    masked_images,
    0.25,
    "../ipynb/masked.gif",
)
del masked_images
gc.collect()

display_gif("../ipynb/masked.gif")

In [ ]:
!cp -a ../data/dataset/ngp/train/. ../data/dataset/ngp/test/

### InstantNGP

In [ ]:
%cd ../ext/instant-ngp

In [ ]:
!python scripts/run.py \
    ../../data/dataset/ngp \
    --n_steps 5000 \
    --save_snapshot ../../data/dataset/ngp_trained.ingp \
    --sharpen 1.0

In [ ]:
!python scripts/run.py \
    --load_snapshot ../../data/dataset/ngp_trained \
    --sharpen 1.0 \
    --video_camera_path ../../data/dataset/ngp/base_cam.json \
    --video_n_seconds 10 \
    --video_fps 24 \
    --width 3072 \
    --height 4096 \
    --video_output ../../data/dataset/ngp_trained.mp4

### Evaluation

In [ ]:
video = cv2.VideoCapture("../../data/dataset/ngp_trained.mp4")
success, image = video.read()
if success:
    cv2.imwrite("../../data/dataset/first_frame.png", image)

In [ ]:
!python evaluation.py --settings_path evaluation.json